In [ ]:
import os
from azureml.core import Workspace
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from torchvision.datasets import CIFAR10
from torchvision import transforms
import random
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import requests

In [ ]:
ws = Workspace.from_config()
ws

In [ ]:
%%writefile pred/score.py

# score.pyの出力

import json
import numpy as np
import onnxruntime
import sys
import os
import time
import torch

from transformers import BertJapaneseTokenizer


def init():
    global session, input_name, output_name, tokenizer
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.onnx')
    session = onnxruntime.InferenceSession(model, None)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name 
    tokenizer = BertJapaneseTokenizer.from_pretrained(
        'cl-tohoku/bert-base-japanese-whole-word-masking')
    

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    input = json.loads(input_data_json)['data']
    print(input)
    input = tokenizer(
            input,
            padding=True,
            max_length=512,
            truncation=True,
            return_tensors="pt")["input_ids"]
    input= input.to('cpu').detach().numpy().copy()
    print(input)
    return input

def postprocess(result):
    # We use argmax to pick the highest confidence label
    return int(np.argmax(np.array(result).squeeze(), axis=0))
    
def run(input_data):

    try:
        # load in our data, convert to readable format
        data = preprocess(input_data)
        
        # start timer
        start = time.time()
        
        r = session.run([output_name], {input_name: data})
        print(r)
        #end timer
        end = time.time()
        
        result = postprocess(r)
        result_dict = {"result": result,
                      "time_in_sec": end - start}
    except Exception as e:
        result_dict = {"error": str(e)}
    
    return result_dict

def choose_class(result_prob):
    """We use argmax to determine the right label to choose from our output"""
    return int(np.argmax(result_prob, axis=0))

In [ ]:
# 推論環境の定義ファイル生成と環境設定

myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-core", "azureml-defaults", "transformers", "fugashi", "ipadic", "torch"])
env_file_path = os.path.join("pred", "environment.yml")
score_file_path = os.path.join("pred", "score.py")

with open(env_file_path, "w") as f:
    f.write(myenv.serialize_to_string())

env = Environment.from_conda_specification(name="onnx_env", file_path=env_file_path)
inference_config = InferenceConfig(entry_script=score_file_path, environment=env)

In [ ]:
# ACI設定
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'framework': 'onnx'}, 
                                               description = 'bert fine-tuned for livedoor news corpus')

In [ ]:
# モデル指定
model = Model(ws, 'bert-livedoor-model')

In [ ]:
# デプロイ
aci_service_name = 'bert-livedoor-api'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
# 推論
endpoint = aci_service.scoring_uri
input_data = json.dumps({'data': ["この副題はどこに分類される？"]})
res = requests.post(url=endpoint, data=input_data, headers={'Content-Type': 'application/json'})
res.json()

In [ ]:
print("予測値: "+str(res.json()["result"]))
#print("正解: "+str(int(target_class)))